  # 0. Bibliotek och data
  Biblioteket `sqlite3` krävs för att skapa funktionen `Nyrad()` nedan. SLOSH-datafilen läses in.

In [1]:
import sqlite3
import pandas as pd

In [2]:
datafil = r'\\win.su.se\dfs\common\stressforskning.su.se\SLOSH-data\SLOSH och AMU\SLOSH\Enkätdata\SLOSH (Huvudfil, t.o.m. 2018)\Test\Slosh_all_amuar_18.xlsx'

df = pd.read_excel(datafil)

# 1. Definiera funktionen Nyrad
Funktionen `Nyrad()` tar (namnet på) en tabell, en tuple med kolumnnamn samt en tuple med kolumnvärden. Den skapar en rad av denna information och försöker lägga in den i databasen. Om den lyckas kommer den SQL-kod som används för att skapa raden att noteras i en text-fil. Denna text-fil ska tjäna dels som en backup men även som en logg. Skulle något hända med databasen går det enkelt att bara köra text-filen så återskapas allting.  
  Ifall funktionen inte lyckas lägga in en rad i databasen, kanske p.g.a. att "foregin key" inte respekteras, kommer ett felmeddelande konstatera detta. För mer information om hur SQLite används i python: https://pynative.com/python-sqlite-insert-into-table/.

In [3]:
def Nyrad(tabell, kolumner, värden):    
    try:
        # Koppla databasen
        conn = sqlite3.connect('SLOSH.db')

        # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
        cursor = conn.cursor()

        # Denna query ska skrivas i SQL och körs av python mot databasen
        sqlite_insert_query = 'INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden)

        # Först måste 'foreign key constraint' slås på, i SQLite är detta avstängt som default. 
        cursor.execute('PRAGMA foreign_keys = 1')

        # Nu kan raden läggas in
        cursor.execute(sqlite_insert_query)

        # commit:a till databasen
        conn.commit()
        print('\033[1m' + 'Rad inlagd i tabellen {}'.format(tabell) + '\033[0m')

        # Stäng
        cursor.close()
        
        print('INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden))
        
        # När en rad lagts till ska SQL-kommandot sparas i en textfil.
        text = open("SQL-logg.txt", "a", encoding = 'UTF-8')
        text.write('INSERT INTO {} {} VALUES {};\n \n'.format(tabell, kolumner, värden))
        text.close()

    except sqlite3.Error as error:
        print('\033[1m' + 'Det gick ej att infoga data: ' + '\033[0m', error)
    finally:
        if (conn):
            conn.close()

    

# 2.Transkribera  
Här transkriberas ett item i taget. Den transkriberade informationen kommer att användas för att skapa rader i flera olika tabeller. Exempelvis förekommer `kodlista` i tre olika tabeller. Se först hur stor andel av SLOSH som redan dokumenterats.    

**OBS!**  
Det är viktigt att databasfilen `SLOSH.db` inte används av något annat program samtidigt som pythonkärnan försöker få tillgång till den. Det går inte att skriva information till databasen från mer än ett ställe samtidigt. Se följande kommentar från stackoverflow:  
>Yes SQLite can support multiple users at once. It does however lock the whole database when writing, so if you have lots of concurrent writes it is not the database you want (usually the time the database is locked is a few milliseconds - so for most uses this does not matter). But it is very well tested and very stable (and widely used) so you can trust it.
You may read this short document for information when to use SQLite and not: http://www.sqlite.org/whentouse.html

## 2.1 Andel av SLOSH som redan har dokumenterats  
Här jämförs antalet dokumenterade variabler i relationsdatabasen med det totala antalet variabler i SLOSH-datafilen. Notera att siffran underskattas eftersom datafilen innehåller registervariabler samt `r_x`-variabler. Dessa kommer aldrig att läggas in i relationsdatabasen.

In [4]:
totvar = len(df.columns)

# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen
cursor.execute('SELECT variabel FROM Variabler')

dokvar = len(set(cursor.fetchall()))

# Stäng
cursor.close()


print('\033[1m' + 'Andel dokumenterad data:' + '\033[0m' + '{}%'.format(round(100*dokvar/totvar, 2)))

Andel dokumenterad data:93.54%


Skapa en excelfil som innehåller alla itemnr, för en given enkät, som redan dokumenterats

In [5]:
# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen
cursor.execute('SELECT variabel, enkät, itemnr, enkättext FROM Variabler')

data = cursor.fetchall()

cursor.close()

variabel = [i[0] for i in data]

enkät = [i[1] for i in data]

itemnr = [i[2] for i in data]

enkättext = [i[3] for i in data]

D = pd.DataFrame(list(zip(variabel, enkät, itemnr, enkättext)), columns = ['Variabel', 'Enkät', 'Itemnr', 'enkättext'])

In [6]:
excelfil = D.loc[D.Enkät == 'SLOSH 18 non-working', :]

excelfil.to_excel('nw18.xlsx')

## <a id = "1">2.2 Har variabeln redan dokumenterats?</a> 

In [29]:
a = 'unpaybur'
b = '_7'
variabel = a+b

In [30]:
v = variabel[-1]

slosh = [i for i in df.columns if i[-1] == str(v)]

# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen
cursor.execute('SELECT variabel FROM Variabler')

# alla variabler som finns i db
dokvar = [i[0] for i in cursor.fetchall()]

# Stäng
cursor.close()

if variabel in dokvar:
    print('\033[1m' + 'Variabeln är redan dokumenterad' + '\033[0m')
else:
    print('\033[1m' + 'Ny variabel!' + '\033[0m')

Variabeln är redan dokumenterad


Se efter ifall en given variabel förekommer i working, non-working eller båda.  
Specificera först variabelnamnet.

In [31]:
r = 'r_{}'.format(variabel[-1])

frek_nw = df.loc[df[r] == 1, variabel].value_counts()

frek_w = df.loc[df[r] == 2, variabel].value_counts()

if len(frek_w) == 0:
    print('\033[1m' + 'Resultat:' + '\033[0m' + '\nVariabeln {} förekommer ej i working'.format(variabel))
    
elif len(frek_nw) == 0:
    print('\033[1m' + 'Resultat:' + '\033[0m' + '\nVariabeln {} förekommer ej i non-working'.format(variabel))
    
else:
    print('\033[1m' + 'Resultat:' + '\033[0m' + '\nVariabeln {} förekommer i working och i non-working'.format(variabel))

    

Resultat:
Variabeln unpaybur_7 förekommer i working och i non-working


## 2.3 Undersök frekvenstabellen och transkribera metadata

In [32]:
r = 'r_{}'.format(variabel[-1])

df.loc[df[r] == 2, variabel].value_counts(dropna = False).sort_index()

1.0      58
2.0    1144
3.0    2417
NaN    7934
Name: unpaybur_7, dtype: int64

In [33]:
r = 'r_{}'.format(variabel[-1])

df.loc[df[r] == 1, variabel].value_counts(dropna = False).sort_index()

1.0        18
2.0       454
3.0      1810
23.0        2
123.0       1
NaN      4003
Name: unpaybur_7, dtype: int64

Efter att ha tagit ställning till vilken (eller vilka) enkäter variabeln förekommer i går det att transkribera resten av informationen. Notera att ifall variabeln förekommer i både working och non-working måste det göras två insättningar relationsdatabasen. 

In [34]:
våg = 2 * int(variabel[-1]) + 4

if våg < 10:
    våg = '0{}'.format(str(våg))
else:
    våg = str(våg)

In [35]:
enkät = 'SLOSH {} working'.format(våg)

In [37]:
enkät = 'SLOSH {} non-working'.format(våg)

In [38]:
beskrivning = ''

itemnr = '22_b'

enkättext = 'Om du arbetar frivilligt, upplever du det som en belastning?'

dubbelkodning = ''

kodsvar = {'1': 'Ja, ofta',
           '2': 'Ja, ibland',
           '3': 'Nej, aldrig',
           '>11': 'slask',
           '.': 'Missing'}
          


print('Enkät: {}'.format(enkät))

Enkät: SLOSH 18 non-working


## 2.4 Finns kodlistan redan?  
För att ta ställning till ifall kodlistan redan förekommer i databasen hämtas först all information i *Kodlistor_stor*. Ifall kodlistan redan finns sätts variabeln `kodlista` lika med det nummer kodlistan redan har i databasen. Om den *inte* finns sätts `kodlista` $ = $ 'NULL'. Ifall SQLite får detta som värde, och kolumnen ifråga är s.k. "AUTOINCREMENT", kommer kodlistan automatiskt få ett nytt heltal som id.

In [39]:
# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen.
cursor.execute('SELECT * FROM Kodlistor_stor')

# Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
rows = cursor.fetchall()

# Stäng databasen.
cursor.close()

kods = []
L = []

# gå igenom varje vektor i listan
for i in range(len(rows)):
    
    # Gäller bara första raden. Detta görs för att undvika index-fel nedan (i-1)
    if len(L) == 0:
        
        # Välj ut kod och svarstext.
        a = (rows[i][1], rows[i][2])
        
        # Lägg i a.
        L.append(a)
    else:
        
        # Ifall k_id skiljer från nuvarnade har loopen nått en ny kodlista. Skapa en dict av innehållet i L och
        # nollställ sedan L inför nästa kodlista.
        if rows[i][0] != rows[i-1][0]:
            d = {j[0]: j[1] for j in L}
            k = rows[i-1][0]
            D = [k, d]
            kods.append(D)
            L = []
        
        if i == (len(rows) - 1):
            a = (rows[i][1], rows[i][2])
            L.append(a)
            d = {j[0]: j[1] for j in L}
            k = rows[i][0]
            D = [k, d]
            kods.append(D)
            break

           
            
        a = (rows[i][1], rows[i][2])
        L.append(a)

        
nykodlista = 'ja'
kodlista = 'NULL'

# Om kodlistan redan finns hämtas dess namn och läggs i variabeln kodlista.
for i in kods:
    if kodsvar == i[1]:
        kodlista = i[0]
        nykodlista = 'nej'
        break
        
        
print('\033[1m' + 'Ny kodlista?' + '\033[0m' + '\n{}'.format(nykodlista))    

Ny kodlista?
nej


## 2.5 Finns svarstexterna redan?  
För att slippa att loopen som lägger in svarstexter nedan avbryts halvvägs igenom av att någon svarstext redan finns i databasen filtreras svarstexterna här. Om de redan är inlagda görs inget försök att lägga in dem igen.

In [40]:
# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen.
cursor.execute('SELECT * FROM Svarstexter')

# Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
rows = cursor.fetchall()

# Stäng databasen.
cursor.close()

# De transkriberade svarstexterna.
svarstexter = list(kodsvar.values())

# rows innehåller vektorer. Här plockat texten ut från vektorerna.
dbsvar = [i[0] for i in rows]

# En lista med de svarstexter som inte redan finns i databasen.
filtsvar = [i for i in svarstexter if i not in dbsvar]

if len(filtsvar) > 0:
    print('\033[1m' + 'Nya svarstexter:' + '\033[0m', filtsvar)
    filtsvar
else:
    print('\033[1m' + 'Inga nya svarstexter' + '\033[0m')

Inga nya svarstexter


# 4. Infoga rader i databasen  
Nu kommer den transkriberade informationen att användas tillsamman med funktionen `Nyrad()` för var och en av tabellerna i databasen.

## C: Kodlistor

In [41]:
tabell = 'Kodlistor'

kolumner = '(kodlista)'

värden = '({})'.format(kodlista)

In [42]:
if kodlista == 'NULL':
    Nyrad(tabell, kolumner, värden)
else:
    print('\033[1m' + 'Ingen ny data!'+ '\033[0m')

Ingen ny data!


## G: Enkättexter

In [43]:
tabell = 'Enkättexter'

kolumner = '(enkättext)'

värden = '(\'{}\')'.format(enkättext)

In [44]:
Nyrad(tabell, kolumner, värden)

Det gick ej att infoga data:  UNIQUE constraint failed: Enkättexter.enkättext


## H: Svarstexter

In [45]:
tabell = 'Svarstexter'

kolumner = '(svarstext)'

In [46]:
if len(filtsvar) > 0:
    for i in filtsvar:
        värden = '(\'{}\')'.format(i)
        Nyrad(tabell, kolumner, värden)
else:
    print('\033[1m' + 'Ingen ny data!'+ '\033[0m')

Ingen ny data!


## D: Kodlistor_stor

In [47]:
tabell = 'Kodlistor_stor'

kolumner = ('KODLISTA', 'KOD', 'SVARSTEXT')

Ifall kodlistan inte tidigare fanns i databasen har den nu lagts längst ner/bak i *Kodlistor* och tilldelats ett nummer. Detta nummer (snarare än "NULL") ska nu användas för att lägga in information i *Kodlistor_stor*.

In [48]:
if kodlista == 'NULL':

    # Koppla databasen
    conn = sqlite3.connect('SLOSH.db')

    # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
    cursor = conn.cursor()

    # Denna query ska skrivas i SQL och körs av python mot databasen.
    cursor.execute('SELECT kodlista FROM Kodlistor')

    # Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
    rows = cursor.fetchall()

    # Stäng databasen.
    cursor.close()

    kodlista = str(rows[-1][0])

In [49]:
kodlista

'471'

In [50]:
if nykodlista == 'ja':
    for i in kodsvar.keys():
        värden = '(\'{}\', \'{}\', \'{}\')'.format(kodlista, i, kodsvar[i])
        #värden = ('NULL', kodlista, i, kodsvar[i])
        Nyrad(tabell, kolumner, värden)
else:
    print('\033[1m' + 'Ingen ny data!'+ '\033[0m')

Ingen ny data!


## B: Variabler

In [51]:
tabell = 'Variabler'

kolumner = ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA')

# värden = '(variabel, enkät, beskrivning, dubbelkodning, itemnr, enkättext, kodlista)

värden = '(\'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\')'.format(variabel, enkät, beskrivning, 
                                                                           dubbelkodning, itemnr, enkättext,
                                                                          kodlista)

In [52]:
Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('unpaybur_7', 'SLOSH 18 non-working', '', '', '22_b', 'Om du arbetar frivilligt, upplever du det som en belastning?', '471')


2020-12-21: **Börja på nw18** 

[Tillbaka till 2.2](#1)

# Saknade/konstiga variabler  
* bedmwork_2
* bedhwork_2  
* bedmfree_2  
* bedhfree_2, på samma sätt för "up" istället för "bed". För de hör variablerna har jag glömt att för itemnr skriva "B" innan t.ex. 15. Detta måste gå att göra enkelt i efterhand.  
* ladder_4 har en väldigt konstig kodlista. Fråga Connie?  
* situatxx_1. Kodboken är dunkel  
* titlepre_1 verkar vara felkodad. Enligt datafilen ska den endast förekomma i working, men enligt enkäten och kodboken ska den endast förekomma i non-working.  
* dosatis_1 ska enligt kodboken förekomma i 2006 non-working, C26_g, men enkättexten stämmer inte.  
* empltype_2 saknar frekvenstabell i sloshdatafilen.  
* confttxt_2 saknar frekvenstabell i sloshdatafilen.  
* bankrupt_3 har i databasen bara två svarsalternativ men i enkäten har den fyra.
* Item 93 i working 2010 går ej att hitta i kodoken. Vad är detta för variabel?  
* dsnewjob_3 non-working har sex koder i sin frekvenstabell men endast 5 koder i kodlistan i enkäten. Variabeln är därför inte inlagd i db-filen.  
* item 11_b i 2010 non-working finns inte i kodboken. Vilken variabel är detta?  
* Det är någonting konstigt med item 23 och 24 i nw 2010. Det finns bara ett variabelnamn i kodboken (takecare).  
* Hittar inte nw 2010 item 63 i kodboken, vad heter variabeln?  
* partpare_4 har 0 och 1 i kodlistan, vad betyder detta?
* Variablerna UrsincopA (ända till UrsincopH) verkar inte finnas i datafilen trots att de står med i kodboken (och i enkäterna såklart).
* conflfam_4 finns i även i non-working men så står det inte i kodboken.  
* jag hittar inte w12 item 103 i kodboken.  
* fråga 92 i non-working 12 finns inte i kodboken
* partime_5 förekommer i nw14 (fast bara missing) trots att den inte gör det enligt kodboken. detsamma gäller 6 och 7.  
* Item 99 i w14 verkar inte finnas i kodboken, hittar inte variabeln.  
* employed_6 (som endast förekommer i nw) har tre svarsalternativ i sin frekvenstabell trots att det bara finns två i enkäten. Detsamma gäller selfempl_6.  
* 86 i nw14 hittar jag inte i kodboken.
  
Det finns förmodligen en kodlista som har kod/text '0-24' men detta är förstås fel, det borde vara '0-23'.